In [1]:
# generate the symmetry-corrected indices for move-based convolution
import sys
import numpy as np
sys.path.append('..')

In [2]:
# test the coefficient generation logic the naive way
cell = 4

In [3]:
# display-only code, to visually check the coeffs are in the correct locations on the board
from neural.neural_ import to_pair, generate_all_moves_by_index, move_convolution_indices
from constants import BOARD_SIZE, BOARD_WIDTH, NUM_BIASES
from math import floor

all_inds, num_coeffs = move_convolution_indices()


num_coeffs -= NUM_BIASES # the first 10 in the above function are biases, don't need them
num_fields = BOARD_SIZE

cell = cell+1
tmp = all_inds[cell]
a = np.zeros([BOARD_WIDTH,BOARD_WIDTH])
for (ind, coeff) in tmp[1:]:
    pair = to_pair(ind)
    a[pair[0],pair[1]] = coeff - NUM_BIASES + 1

print(to_pair(cell))
print(a)

(1, 1)
[[ 0.  0.  0.  9.]
 [ 0.  3.  0.  0.]
 [ 0.  0.  0.  8.]
 [ 9.  0.  8.  0.]]


In [4]:
# try calling conv_stack directly from Tensorflow
import numpy as np
import tensorflow as tf

tf.reset_default_graph()

from neural.tensorflow_utils import conv_stack

with tf.Session() as sess:
    in_fields_np = np.ones([2,num_fields])
    in_fields_np[0,3] = 0
    in_fields_np[0,5] = 0
    my_pos = np.array([12, 12])
    other_pos =  np.array([5,5])
    inputs_np = np.concatenate([in_fields_np, my_pos[:,None], other_pos[:,None]],
                              1)
    #print(inputs_np.shape)
    inputs =tf.constant(inputs_np)# tf.placeholder(shape =[None, num_fields+2], dtype = tf.float32) #
    #print(sess.run(inputs))
    out = conv_stack(inputs, 5,sess)
    
    dummy = np.array([50,50])[:,None]
    #print(sess.run(get_random_index(inputs, tf.constant(dummy))))
    
    sess.run(tf.global_variables_initializer())
    stack_result = sess.run(out)#, feed_dict={inputs:inputs_np})
    print(stack_result)

[2 3 2]
[[[ 0.34545547 -0.        ]
  [-0.70719635 -0.        ]
  [ 0.09106012 -0.        ]]

 [[ 0.19336556 -2.59213209]
  [-3.24466944 -3.14675403]
  [-3.75175548 -9.7152338 ]]]


In [5]:
# load game simulation data
import glob
import sys
import pickle
from data_utils import load_simulation_data

fn = '../data/states.pickle'
if True:
# try:
#     with open(fn, 'rb') as f:
#         states = pickle.load(f)
# except:
    files = glob.glob('../data/4x4tiny.pickle')
    #files = glob.glob('../data/epsgreedy/*')
    #files = glob.glob('../data/ID_x2_1000ms/result_ID*.pickle')
    #print(files)
    depths =load_simulation_data(files)
    keys = list(depths.keys())
    #print(keys)
    games = depths[keys[0]]
    #print(games[0])
    states = [state for game in games for state in game if 'score' in state] 
    print(len(states))
    with open(fn, 'wb') as f:
        pickle.dump(states,f)

KeyboardInterrupt: 

In [ ]:
from data_utils import prepare_data_for_model
board_full, player_pos,legal_moves,next_move,y = prepare_data_for_model(states,'simple_score')
print(board_full.shape, player_pos.shape, y.shape)
y[y ==float('-inf')] = 0
y[y ==float('inf')] = 0

In [ ]:
# fit the naive score as a first test of our network
from neural.keras_utils import deep_model_fun
model, Q_model = deep_model_fun(num_features = 8, num_res_modules = 2, drop_rate = 0.01, activation = 'linear')
model.summary()
model.compile(optimizer = 'adam',  loss='mean_squared_error')

In [ ]:
model.fit(x = [board_full, player_pos,legal_moves,next_move],y = y, batch_size = 64, epochs=10, verbose =1)
#Q_model.predict( [board_full, player_pos,legal_moves])[0]

In [ ]:
states[0]

In [ ]:
test = [s for s,state in enumerate(states) if 'score' not in state ]
print(test)

In [ ]:
# Now let's get all those games where tree search actually completed
import numpy as np
from sklearn.preprocessing import OneHotEncoder
complete_states = [state for state in states]# if state['score'] == float('inf') or state['score'] == float('-inf')]
print(len(complete_states))
board_full_c, player_pos_c,legal_moves_c,next_move_c, y_c = prepare_data_for_model(complete_states,'G')
# y_c[y_c==float('inf')] = 1
# y_c[y_c==float('-inf')] = 0

In [ ]:
print(set(list(np.reshape(y_c,[-1]))))

In [ ]:
from neural.keras_utils import deep_model_fun
# width AND depth matter! (32, 8 seem about optimal on this dataset)
deep_model, deep_Q_model = deep_model_fun(num_features =32, num_res_modules = 8, drop_rate = 0.1, activation = 'sigmoid')
deep_model.summary()
#deep_model.compile(optimizer = 'adam',  loss='binary_crossentropy', metrics =['acc'])deep_model.compile(optimizer = 'adam',  loss='binary_crossentropy', metrics =['acc'])
deep_model.compile(optimizer = 'adam',  loss='mean_squared_error', metrics =['acc'])

In [ ]:
deep_model.fit([board_full_c, player_pos_c,legal_moves_c,next_move_c], y_c, batch_size = 128, epochs=10, verbose =1, validation_split = 0.2, shuffle = True)

In [ ]:
deep_Q_model.save_weights('../data/deep_Q_model_weights.h5')

In [ ]:
%load_ext autoreload
%autoreload 2
from collections import namedtuple
from copy import copy
from neural.neural_ import get_legal_moves
from neural.neural_agent import apply_move, get_best_move_from_model
from data_utils import prepare_data_for_model

board = np.ones(BOARD_SIZE)
#print(list(board))
board.sum()
my_pos = None
other_pos = None
game = {'pos': np.array([my_pos, other_pos]), 'game': board, 'move': 0}
game1 = apply_move(game, 0)
game2 = apply_move(game1, 1)
game3 = apply_move(game2,get_legal_moves(game2)[0])
# board, pos, _ = possible_moves_for_model(game3)
# print(board.shape, pos.shape)
get_best_move_from_model(game3, deep_Q_model)
#deep_Q_model.summary()

In [ ]:
from neural.neural_agent import NeuralAgent
my_agent = NeuralAgent(deep_Q_model)
my_agent.get_move(game3)


In [ ]:
from tournament import tournament, Agent, RandomPlayer
from neural.neural_agent import NeuralAgent

my_agent = NeuralAgent(deep_Q_model)
tournament(num_matches=10, time_limit=float('inf'), 
           test_agents=[Agent(my_agent,"Neural Agent")])
a=1